In [1]:
# pip install scrapy
# pip install --upgrade zope2

from collections import Counter
import foursquare
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import random
import requests
from scrapy import Selector
from scrapy.http import HtmlResponse
import seaborn as sns
from sqlalchemy import create_engine
import sqlite3
import textacy
import textacy.data
import time
import unicodedata

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
usda = pd.read_pickle('../../../../Accounts/Electo:Computad/\
MacBook Pro/Software/Github/Old_repos/DSI-SF-4-anotherguitar/projects/Capstone Stuff/usda.pkl')
food = pd.read_pickle('../../../../Accounts/Electo:Computad/\
MacBook Pro/Software/Github/Old_repos/DSI-SF-4-anotherguitar/projects/Capstone Stuff/explore_endpoint_df.pkl')
vegan = food[food.vegan_venue_check == 'vegan']

In [4]:
keyword = 'bakery'
if keyword.lower() in ', '.join(usda.Description.values).lower():
    print "true"

true


In [5]:
usda.head()

,Food_No,Food_Group,Description,Nutr_No,Nutrient,Amount_per_100,Units,Tagname,Not_Imputed,ComName,FG_Cd
0,01001,Dairy and Egg Products,"Butter, salted",203,Protein,0.85,g,PROCNT,1,,0100
1,01001,Dairy and Egg Products,"Butter, salted",204,Total lipid (fat),81.11,g,FAT,1,,0100
2,01001,Dairy and Egg Products,"Butter, salted",205,"Carbohydrate, by difference",0.06,g,CHOCDF,0,,0100
3,01001,Dairy and Egg Products,"Butter, salted",207,Ash,2.11,g,ASH,1,,0100
4,01001,Dairy and Egg Products,"Butter, salted",208,Energy,717.00,kcal,ENERC_KCAL,0,,0100


In [6]:
# usda = usda.drop('ComName', axis=1)
# usda = usda.drop('FG_Cd', axis=1)

In [7]:
usda.head()

,Food_No,Food_Group,Description,Nutr_No,Nutrient,Amount_per_100,Units,Tagname,Not_Imputed
0,01001,Dairy and Egg Products,"Butter, salted",203,Protein,0.85,g,PROCNT,1
1,01001,Dairy and Egg Products,"Butter, salted",204,Total lipid (fat),81.11,g,FAT,1
2,01001,Dairy and Egg Products,"Butter, salted",205,"Carbohydrate, by difference",0.06,g,CHOCDF,0
3,01001,Dairy and Egg Products,"Butter, salted",207,Ash,2.11,g,ASH,1
4,01001,Dairy and Egg Products,"Butter, salted",208,Energy,717.00,kcal,ENERC_KCAL,0


In [8]:
columns = usda.Food_Group.unique().ravel()

In [9]:
dads = usda[['Food_Group','Description']]
# descriptions = dads.Description.unique()
# for i in descriptions:
#     identity = usda.usda.Description == i
dads.head()

,Food_Group,Description
0,Dairy and Egg Products,"Butter, salted"
1,Dairy and Egg Products,"Butter, salted"
2,Dairy and Egg Products,"Butter, salted"
3,Dairy and Egg Products,"Butter, salted"
4,Dairy and Egg Products,"Butter, salted"


In [10]:
dads['Food_Group'][1]

'Dairy and Egg Products'

In [11]:
dads['Description'][1]

'Butter, salted'

In [16]:
vegan_next_version = vegan
list_run_thru = vegan_next_version.menu_item_name.unique()
list_run_thru

array(["Tokaji, Disznoko, Kesoi Szuret, Hungary '08",
       "Tawny Port, Graham's 20 Yr", 'Porto, Rozes, 1997', ...,
       'Coke / Diet Coke', 'Kettle Chips', 'Just Baked All-Natural Cookies'], dtype=object)

In [17]:
for i in list_run_thru:
    i.split()
    if len(i.split()) == 1:
        print 'one word'
    else:
        print '>1 words'

>1 words
>1 words
>1 words
>1 words
>1 words
>1 words
>1 words
>1 words
>1 words
>1 words
>1 words
>1 words
>1 words
>1 words
>1 words
>1 words
>1 words
>1 words
>1 words
>1 words
>1 words
>1 words
>1 words
>1 words
>1 words
>1 words
>1 words
>1 words
>1 words
>1 words
one word
>1 words
>1 words
>1 words
>1 words
>1 words
>1 words
>1 words
>1 words
>1 words
>1 words
>1 words
>1 words
>1 words
>1 words
>1 words
>1 words
>1 words
>1 words
>1 words
>1 words
>1 words
>1 words
>1 words
>1 words
>1 words
>1 words
>1 words
>1 words
>1 words
>1 words
>1 words
>1 words
>1 words
>1 words
one word
>1 words
>1 words
>1 words
>1 words
>1 words
>1 words
>1 words
>1 words
>1 words
>1 words
>1 words
>1 words
>1 words
one word
>1 words
>1 words
>1 words
>1 words
>1 words
>1 words
>1 words
>1 words
>1 words
>1 words
>1 words
>1 words
>1 words
>1 words
>1 words
>1 words
>1 words
>1 words
>1 words
>1 words
>1 words
>1 words
>1 words
>1 words
one word
>1 words
>1 words
>1 words
>1 words
>1 words
>1 words
>

In [18]:
deduped = []
for group, group_df in dads.groupby(['Food_Group', 'Description']):
    deduped.append({
      "Food_Group":  group[0],
      "Description": group[1]
    })
    
# pd.DataFrame(deduped)
keyword = 'Butter'
in_group = []
for i in range(len(usda.Description.unique())):
    if keyword.lower() in deduped[i]['Description']:
        i_desc = deduped[i]['Food_Group'].lower().replace(',',' ').replace('/',' ')
        i_desc = i_desc.replace('-',' ').replace('_',' ').replace('.',' ').replace(',',' ')
        in_group.append(i_desc)
print list(set(in_group))
print len(list(set(in_group)))

['fast foods', 'soups  sauces  and gravies', 'snacks', 'finfish and shellfish products', 'dairy and egg products', 'legumes and legume products', 'breakfast cereals', 'vegetables and vegetable products', 'sweets', 'baby foods', 'nut and seed products', 'fats and oils', 'baked products']
13


In [19]:
deduped[2]['Food_Group'].lower().replace(',',' ').replace('/',' ').replace('-',' ').replace('_',' ').replace('.',' ').replace(',',' ') 

'american indian alaska native foods'

In [20]:
usda.shape

(595359, 9)

In [21]:
tmp = usda[['Food_Group','Nutrient']]

In [22]:
tmp = tmp.groupby(['Food_Group','Nutrient']).agg(lambda x: 1.)
tmp.head()

Food_Group                           Nutrient
American Indian/Alaska Native Foods  10:0        1.0
                                     12:0        1.0
                                     14:0        1.0
                                     14:1        1.0
                                     15:0        1.0
dtype: float64

In [23]:
tmp = tmp.unstack(fill_value=0)

In [24]:
tmp.shape

(25, 145)

In [25]:
tmp = tmp.reset_index()

In [26]:
tmp['Carbohydrate, by difference']

0     1.0
1     1.0
2     1.0
3     1.0
4     1.0
5     1.0
6     1.0
7     1.0
8     1.0
9     1.0
10    1.0
11    1.0
12    1.0
13    1.0
14    1.0
15    1.0
16    1.0
17    1.0
18    1.0
19    1.0
20    1.0
21    1.0
22    1.0
23    1.0
24    1.0
Name: Carbohydrate, by difference, dtype: float64

In [27]:
tmp.columns.unique()

array(['Food_Group', '10:0', '12:0', '13:0', '14:0', '14:1', '15:0',
       '15:1', '16:0', '16:1 c', '16:1 t', '16:1 undifferentiated', '17:0',
       '17:1', '18:0', '18:1 c', '18:1 t', '18:1 undifferentiated',
       '18:1-11t (18:1t n-7)', '18:2 CLAs', '18:2 i', '18:2 n-6 c,c',
       '18:2 t not further defined', '18:2 t,t', '18:2 undifferentiated',
       '18:3 n-3 c,c,c (ALA)', '18:3 n-6 c,c,c', '18:3 undifferentiated',
       '18:3i', '18:4', '20:0', '20:1', '20:2 n-6 c,c', '20:3 n-3',
       '20:3 n-6', '20:3 undifferentiated', '20:4 n-6',
       '20:4 undifferentiated', '20:5 n-3 (EPA)', '21:5', '22:0', '22:1 c',
       '22:1 t', '22:1 undifferentiated', '22:4', '22:5 n-3 (DPA)',
       '22:6 n-3 (DHA)', '24:0', '24:1 c', '4:0', '6:0', '8:0',
       'Adjusted Protein', 'Alanine', 'Alcohol, ethyl', 'Arginine', 'Ash',
       'Aspartic acid', 'Beta-sitosterol', 'Betaine', 'Caffeine',
       'Calcium, Ca', 'Campesterol', 'Carbohydrate, by difference',
       'Carotene, alpha', 'C

In [28]:
tmp['Sucrose'].head(3).T

0    1.0
1    1.0
2    1.0
Name: Sucrose, dtype: float64

In [29]:
tmp.head(10)

Nutrient,Food_Group,10:0,12:0,13:0,14:0,14:1,15:0,15:1,16:0,16:1 c,...,"Vitamin C, total ascorbic acid",Vitamin D,Vitamin D (D2 + D3),Vitamin D2 (ergocalciferol),Vitamin D3 (cholecalciferol),Vitamin E (alpha-tocopherol),"Vitamin E, added",Vitamin K (phylloquinone),Water,"Zinc, Zn"
0,American Indian/Alaska Native Foods,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0
1,Baby Foods,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,...,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
2,Baked Products,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
3,Beef Products,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
4,Beverages,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0
5,Breakfast Cereals,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
6,Cereal Grains and Pasta,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0
7,Dairy and Egg Products,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
8,Fast Foods,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
9,Fats and Oils,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
